<a href="https://colab.research.google.com/github/kivilcimm/Using-Machine-Learning-for-Predicting-Software-Size-from-Requirements/blob/main/deneme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

from google.colab import drive

import pandas as pd
import numpy as np
import random
from datetime import datetime

import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup



In [ ]:
try:
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/Dataset/"

except:
    print("You are not working in Colab at the moment :(")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#path = data_path + "cosmic_update.csv"
path = data_path + "F_cosmic.csv"
trng_dataset = pd.read_csv(path , delimiter=',')
print('Number of training sentences =', trng_dataset.shape[0])
trng_dataset.head()


Number of training sentences = 2602


,projectNo,usecaseNo,sentence,totalCFP
0,1,0.0,User fills in the required input fields provid...,1
1,1,NaN,He/She clicks on the ‘Register’ button.,0
2,1,NaN,System will ask the user to provide his/her ph...,3
3,1,NaN,Then the user will be directed to a page in wh...,0
4,1,NaN,System will give 2 minutes for the user to ent...,1


In [ ]:
trng_dataset.shape

(2602, 4)

In [ ]:
trng_sentences = trng_dataset.sentence.values
trng_labels = trng_dataset.totalCFP.values
trng_labels = trng_labels.astype('float32')


trng_project_no = trng_dataset.projectNo.values#****
trng_usecase_no = trng_dataset.usecaseNo.values#*****


In [ ]:
trng_labels.dtype

dtype('float32')

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
max_len = 0

for sent in trng_sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    #print("input_ids: ",input_ids)
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)


Max sentence length:  106


In [ ]:
input_ids = []
attention_masks = []

for sent in trng_sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 100,           
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )
 
    input_ids.append(encoded_dict['input_ids'])
  #  print("input_ids: ",input_ids)
    attention_masks.append(encoded_dict['attention_mask'])
 #   print("attention_masks: ",attention_masks)

In [ ]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(trng_labels, dtype=torch.float) 
print('Original: ', trng_sentences[0])
print('Token IDs:', input_ids[0])



Original:  User fills in the required input fields provided by the system with their information.
Token IDs: tensor([  101,  5310, 17469,  1999,  1996,  3223,  7953,  4249,  3024,  2011,
         1996,  2291,  2007,  2037,  2592,  1012,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])


In [ ]:
dataset = TensorDataset(input_ids, attention_masks, labels)
print(dataset[0][0])
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
print(train_dataset)
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))


tensor([  101,  5310, 17469,  1999,  1996,  3223,  7953,  4249,  3024,  2011,
         1996,  2291,  2007,  2037,  2592,  1012,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
2,081 training samples
  261 validation samples
  260 test samples


In [ ]:
batch_size = 32

train_dataloader = DataLoader(
            train_dataset, 
            shuffle = True,
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

test_dataloader = DataLoader(
            test_dataset,
          # sampler = SequentialSampler(test_dataset), 
            batch_size = batch_size 
        )

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",      
    num_labels = 1,                 
    output_attentions = False,      
    output_hidden_states = False,   
)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, 
                  eps = 1e-8 
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 4

In [ ]:
def get_mse(preds, labels):
    pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    return np.sum((pred_flat - labels_flat)**2)/len(labels_flat)

def get_mae(preds, labels):
    pred_flat = preds.flatten()
    labels_flat = labels.flatten()

    sum = 0
    n = len(labels_flat)
    
    for i in range(n):
      
        temp = abs(pred_flat[i] - labels_flat[i])
        sum += temp

    error = sum/n 
    print("Mean absolute error : " + str(error))

    return error

In [ ]:
seed_val = 20

random.seed(seed_val)
np.random.seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
t_start = datetime.now()
d = {}

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')


    t_epoch_trng = datetime.now()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = (datetime.now() - t_epoch_trng).total_seconds()
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        trng_output = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
                
        total_train_loss += trng_output.loss.item()

        trng_output.loss.backward()

        optimizer.step()
        print("train_dataloader: ",trng_output)
    avg_train_loss = total_train_loss / len(train_dataloader)            

    training_time = datetime.now() - t_epoch_trng

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: ", training_time.total_seconds())

    print("")
    print("Running Validation...")

    t_epoch_validation = datetime.now()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        print("b_input_ids: ", b_input_ids)
        print("b_input_ids: ", b_input_mask)


        with torch.no_grad():        

            validation_output = model(b_input_ids, 
                                   attention_mask=b_input_mask)

        logits = validation_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        print("logits: ",logits.flatten())
        print("label_ids",label_ids.flatten())
        total_eval_accuracy += get_mae(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = datetime.now() - t_epoch_validation
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: ", validation_time.total_seconds())
    d[epoch_i] = model.cuda()
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("Total training took: ", (datetime.now() - t_start).total_seconds())


Görüntülenen çıkış son 5000 satıra kısaltıldı.
        [ 0.9940]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
train_dataloader:  SequenceClassifierOutput(loss=tensor(0.0260, device='cuda:0', grad_fn=<MseLossBackward0>), logits=tensor([[0.1614]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

  Average training loss: 0.46
  Training epoch took:  30.39428

Running Validation...
b_input_ids:  tensor([[  101,  2330,  2689,  ...,     0,     0,     0],
        [  101, 12210,  4471,  ...,     0,     0,     0],
        [  101, 11562,  5587,  ...,     0,     0,     0],
        ...,
        [  101,  4653,  2035,  ...,     0,     0,     0],
        [  101,  1996,  2291,  ...,     0,     0,     0],
        [  101,  1996,  2291,  ...,     0,     0,     0]], device='cuda:0')
b_input_ids:  tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0,

In [ ]:
def evaluate(model, test_dataset, device):

    test = test_dataset
    print("test_dataset: ",test_dataset[0][0])
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

        total_acc_test = 0
        logits_arr = []
        label_ids_arr = []
        logits_ = []
        labels_ = []
        with torch.no_grad():

            for batch in test_dataloader:
        
               b_input_ids = batch[0].to(device)
               b_input_mask = batch[1].to(device)
               b_labels = batch[2].to(device)      

               test_output = model(b_input_ids, 
                                   attention_mask=b_input_mask)
    
               logits = test_output.logits.detach().cpu().numpy()
               label_ids = b_labels.to('cpu').numpy()
               total_acc_test += get_mae(logits, label_ids)
               
               logits_arr = logits.flatten()
               label_ids_arr = label_ids.flatten()
               #print("Pred: ", logits_arr)
               #print("Actual: ",label_ids_arr)
               for l in logits_arr:
                 logits_.append(l)
               for la in label_ids_arr:
                 labels_.append(la)
               print("Pred: ", logits_)
               print("Actual: ",labels_)  


    avg_test_accuracy = total_acc_test / len(test_dataloader)
    print("Test Accuracy: {0:.3f}".format(avg_test_accuracy))
    total = 0
    x = 0
    c = 0
    for j in logits_:
        if(labels_[x] != 0.0):
          total += abs(j-labels_[x]) / abs(labels_[x])
          c += 1
          x = x + 1
          #print("Total: ", total)
          print("c: ", c)
        else:
          x = x + 1
          continue
          
    average = total / c   
    print("MRE value: ", total) 
    print("MMRE value: ", average) 

    p = 0
    c1 = 0
    c2 = 0
    j = 0
    x = 0
    for j in logits_:
        if(labels_[x] != 0.0):
          p = abs(j-labels_[x]) / abs(labels_[x])
          print("p: ",p)
          c1 += 1
          if (p < 0.3):
            c2 += 1
            
          x = x + 1
          p = 0
          #print("Total: ", total)
          print("c: ", c1)
        else:
          x = x + 1
          continue
    print("Total Number for PRED30: ", c2)
    print("PRED30: ", (c2/c1))   

valid_accuracy = [x['Valid. Accur.'] for x in training_stats]
print("Valid_accuracy: ", valid_accuracy)
print("min index: ",np.argmin(valid_accuracy))
min = np.argmin(valid_accuracy)

model = d[min]
data_p = "/content/drive/My Drive/Dataset/record.pt"
torch.save(model, data_p)
evaluate(model, test_dataset, device)



Valid_accuracy:  [0.45778720764081093, 0.37920105682711036, 0.3519629239104688, 0.34138709328148653]
min index:  3
test_dataset:  tensor([  101,  8619, 12040,  2015,  2010,  1013,  2014,  8833,  2378,  2592,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Mean absolute error : 0.3528059867094271
Pred:  [1.9779304, 1.8831135, 1.1456013, 1.24040